In [1]:
import numpy as np

In [2]:
MOUNT_PATH = "/run/user/1000/gvfs/smb-share:server=azlab-fs01,share=東研究室/個人work/富井/"

In [3]:
ls $MOUNT_PATH

20170807RFdata/  2017-11-24_1/  2018-01-13_1/       PYUSCT_model/
2017-11-20/      2017-12-18_1/  2018-03-30/         PYUSCT_train/
2017-11-20_2/    2018-01-10_1/  2018-04-01/         test.txt*
2017-11-22_1/    2018-01-10_2/  nb_opmap_rayleigh/  training_data.hdf5*
2017-11-23_1/    2018-01-12_1/  nb_usctsim/
2017-11-23_2/    2018-01-12_2/  optical/


In [4]:
DATA_PATH = MOUNT_PATH + "PYUSCT_train/dataset008/" 

In [5]:
ls $DATA_PATH

dataset.npz*              image/     output/      sa.npy*
full_visual/              input/     param.json*  sa_small.npy*
full_visual_t100_r64/     list.csv*  pca_data/    sa_small.png*
full_visual_T8_t100_r64/  ml_model/  sa/


In [6]:
X = np.load(DATA_PATH + "pca_data/X_pca_sample10000_reduced_t100_r64.npy")
y = np.load(DATA_PATH + "pca_data/y_pca_sample10000_reduced_t100_r64.npy")

In [7]:
X.shape, y.shape

((10000, 800), (10000,))

## Data split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
X_train.shape, X_test.shape

((6700, 800), (3300, 800))

## Classification

In [11]:
from sklearn.metrics import classification_report

In [12]:
target_names = ["no edge", "edge"]

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_l1_LR = LogisticRegression(C=20, penalty='l1', tol=0.01)
clf_l2_LR = LogisticRegression(C=20, penalty='l2', tol=0.01)

#### with L1

In [ ]:
clf_l1_LR.fit(X_train, y_train)

In [ ]:
y_pred = clf_l1_LR.predict(X_test)

In [ ]:
print("Logistic with L1")
print(classification_report(y_test, y_pred, target_names=target_names))

#### with L2

In [ ]:
clf_l2_LR.fit(X_train, y_train)

In [ ]:
y_pred = clf_l2_LR.predict(X_test)

In [ ]:
print("Logistic with L2")
print(classification_report(y_test, y_pred, target_names=target_names))

### SVM/RVM

In [ ]:
from sklearn import svm

In [ ]:
clf_svm_rbf = svm.SVC(kernel='rbf')
clf_svm_linear = svm.SVC(kernel='linear')
clf_svm_sigmoid = svm.SVC(kernel='sigmoid')

#### rbf kernel

In [ ]:
clf_svm_rbf.fit(X_train, y_train)

In [ ]:
y_pred = clf_svm_rbf.predict(X_test)

In [ ]:
print("SVM with rbf kernel")
print(classification_report(y_test, y_pred, target_names=target_names))

#### linear kernel

In [ ]:
clf_svm_linear.fit(X_train, y_train)

In [ ]:
y_pred = clf_svm_linear.predict(X_test)

In [ ]:
print("SVM with linear kernel")
print(classification_report(y_test, y_pred, target_names=target_names))

#### sigmoid kernel

In [ ]:
clf_svm_sigmoid.fit(X_train, y_train)

In [ ]:
y_pred = clf_svm_sigmoid.predict(X_test)

In [ ]:
print("SVM with sigmoid kernel")
print(classification_report(y_test, y_pred, target_names=target_names))

### Gaussian Process Regression

In [ ]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [ ]:
gp_kernel = ExpSineSquared(1.0, 5.0, periodicity_bounds=(1e-2, 1e1)) \
    + WhiteKernel(1e-1)
iso_kernel = 1.0 * RBF([1.0])

In [ ]:
gpr = GaussianProcessRegressor(kernel=iso_kernel)

In [ ]:
y_train.dtype

In [ ]:
gpr.fit(X_train, y_train)

In [ ]:
y_pred = gpr.predict(X_test)

In [ ]:
y_pred[:10]

In [ ]:
y_test[:10]

In [ ]:
from sklearn.metrics import explained_variance_score
print("Gaussian Process Regression with isotropic rbf kernel")
print(explained_variance_score(y_test, y_pred))

### Gaussian Process Classification

In [13]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier

#### Isotropic kernel

In [14]:
clf_GPC_rbf_iso = GaussianProcessClassifier(1.0 * RBF([1.0]) )

In [15]:
clf_GPC_rbf_iso.fit(X_train, y_train)

GaussianProcessClassifier(copy_X_train=True,
             kernel=1**2 * RBF(length_scale=1), max_iter_predict=100,
             multi_class='one_vs_rest', n_jobs=1, n_restarts_optimizer=0,
             optimizer='fmin_l_bfgs_b', random_state=None,
             warm_start=False)

In [16]:
y_pred = clf_GPC_rbf_iso.predict(X_test)

In [17]:
print("Gaussian Process Classification with isotropic rbf kernel")
print(classification_report(y_test, y_pred, target_names=target_names))

Gaussian Process Classification with isotropic rbf kernel
             precision    recall  f1-score   support

    no edge       0.97      0.87      0.92      1564
       edge       0.89      0.97      0.93      1736

avg / total       0.93      0.92      0.92      3300



In [18]:
import pickle
from sklearn.externals import joblib
from sklearn.decomposition import PCA

clf_GPC_rbf_iso = joblib.load(DATA_PATH + "clf_GPC_rbf_iso_t100_r64.pkl")
print("model loaded.")

FileNotFoundError: [Errno 2] No such file or directory: '/run/user/1000/gvfs/smb-share:server=azlab-fs01,share=東研究室/個人work/富井/PYUSCT_train/dataset008/clf_GPC_rbf_iso_t100_r64.pkl'

#### Anisotropic rbf kernel

In [ ]:
del pca

In [ ]:
weights = pca.explained_variance_ratio_
weights.shape

In [ ]:
clf_GPC_rbf_ani = GaussianProcessClassifier(1.0 * RBF(weights))

In [ ]:
clf_GPC_rbf_ani.fit(X_train, y_train)

In [ ]:
y_pred = clf_GPC_rbf_ani.predict(X_test)

In [ ]:
print("Gaussian Process Classification with anisotropic rbf kernel")
print(classification_report(y_test, y_pred, target_names=target_names))

### Ensemble method

####  Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_RF = RandomForestClassifier()

In [ ]:
clf_RF.fit(X_train, y_train)

In [ ]:
y_pred = clf_RF.predict(X_test)

In [ ]:
print("Random Forest Classifier")
print(classification_report(y_test, y_pred, target_names=target_names))

#### Adboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf_Adb = AdaBoostClassifier()

In [ ]:
clf_Adb.fit(X_train, y_train)

In [ ]:
y_pred = clf_Adb.predict(X_test)

In [ ]:
print("AdaBoost Classifier")
print(classification_report(y_test, y_pred, target_names=target_names))

#### XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
clf_XGb = XGBClassifier()

In [ ]:
clf_XGb.fit(X_train, y_train)

In [ ]:
y_pred = clf_XGb.predict(X_test)

In [ ]:
print("XGBoost Classifier")
print(classification_report(y_test, y_pred, target_names=target_names))

### Load model

In [ ]:
from sklearn.externals import joblib

In [ ]:
model_clf= joblib.load("clf_GPC_rbf_iso.pkl")

### Save model

In [19]:
from sklearn.externals import joblib

In [20]:
# FPATH = MOUNT_PATH + "PYUSCT_model/"
FPATH = DATA_PATH

In [21]:
model_clf = clf_GPC_rbf_iso
model_name = "clf_GPC_rbf_iso"

In [22]:
joblib.dump(model_clf, FPATH + model_name + "_t100_r64.pkl") 

['/run/user/1000/gvfs/smb-share:server=azlab-fs01,share=東研究室/個人work/富井/PYUSCT_train/dataset008/clf_GPC_rbf_iso_t100_r64.pkl']